In [2]:
import torch
import torch.nn as nn

# Definir la clase EmotionRecognitionModel antes de cargar el modelo
class EmotionRecognitionModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(EmotionRecognitionModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        h_0 = torch.zeros(self.lstm.num_layers, x.size(0), self.lstm.hidden_size).to(x.device)
        c_0 = torch.zeros(self.lstm.num_layers, x.size(0), self.lstm.hidden_size).to(x.device)
        
        out, _ = self.lstm(x, (h_0, c_0))
        out = out[:, -1, :]  # Tomar la salida del último timestep
        out = self.fc(out)
        return out

# Cargar el modelo completo
model = torch.load('modelo_emociones_lstm_completo.pth')



/var/folders/pv/hj1tkdtn4xd2lk842kp9xy7r0000gn/T/ipykernel_4932/4264266241.py:21: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load('modelo_emociones_lstm_com

In [ ]:
import torch
import numpy as np
from collections import deque

# Asegúrate de que el modelo esté en modo de evaluación y en el dispositivo correcto
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.eval()

# Configuración de la secuencia de tiempo (timesteps)
n_timesteps = 101  # Debe coincidir con lo que usaste para entrenar el modelo
n_features = 38  # Asegúrate de que esto coincida con el número de características por timestep usado en el entrenamiento

# Inicializar un buffer para mantener las secuencias de características en tiempo real
sequence_buffer = deque(maxlen=n_timesteps)

# Inicializar la emoción previa para detección de cambios
previous_emotion = None

# Simulación de obtención de características en tiempo real (ajusta para que devuelva 38 características)
def get_real_time_features():
    # Esta función debería devolver un vector de características de longitud `n_features` (38 en este caso)
    # Aquí simplemente devolvemos datos aleatorios como ejemplo
    return np.random.rand(n_features)

# Ejemplo de bucle de predicción en tiempo real
while True:
    # Obtener nuevas características en tiempo real
    new_features = get_real_time_features()

    # Añadir las nuevas características al buffer
    sequence_buffer.append(new_features)

    # Verificar si hemos llenado el buffer con suficientes timesteps
    if len(sequence_buffer) == n_timesteps:
        # Convertir el buffer a un tensor y hacer la predicción
        sequence_array = np.array(sequence_buffer)
        input_tensor = torch.tensor(sequence_array, dtype=torch.float32).unsqueeze(0).to(device)  # Añadir dimensión de batch

        with torch.no_grad():
            output = model(input_tensor)
            _, predicted = torch.max(output, 1)
            current_emotion = predicted.item()

        # Detectar cambio de emoción
        if current_emotion != previous_emotion:
            print(f'Emoción cambiada: {current_emotion}')
            previous_emotion = current_emotion
            # Aquí podrías agregar lógica para mostrar la nueva emoción, guardarla, etc.

    # Simulación de esperar un poco antes de la siguiente iteración (ej. 100ms)
    # Para tiempos reales, esta parte dependería de cómo obtienes las características en tiempo real
    # time.sleep(0.1)


In [11]:
import torch
import numpy as np
from collections import deque, Counter
import time

# Asegúrate de que el modelo esté en modo de evaluación y en el dispositivo correcto
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.eval()

# Configuración de la secuencia de tiempo (timesteps)
n_timesteps = 101
n_features = 38

# Inicializar un buffer para mantener las secuencias de características en tiempo real
sequence_buffer = deque(maxlen=n_timesteps)

# Ventana deslizante para suavizar predicciones
prediction_window = deque(maxlen=5)  # Ventana de las últimas 5 predicciones

# Inicializar la emoción previa para detección de cambios
previous_emotion = None

# Función para simular la obtención de características en tiempo real
def get_real_time_features():
    raw_features = np.random.rand(n_features)
    # Aplica el mismo preprocesamiento que usaste en el entrenamiento
    normalized_features = (raw_features - np.mean(raw_features)) / np.std(raw_features)
    return normalized_features

# Función para calcular la latencia
def calculate_latency(start_time):
    return time.time() - start_time

# Bucle de predicción en tiempo real
while True:
    start_time = time.time()

    new_features = get_real_time_features()
    sequence_buffer.append(new_features)

    if len(sequence_buffer) == n_timesteps:
        sequence_array = np.array(sequence_buffer)
        input_tensor = torch.tensor(sequence_array, dtype=torch.float32).unsqueeze(0).to(device)

        with torch.no_grad():
            output = model(input_tensor)
            _, predicted = torch.max(output, 1)
            current_emotion = predicted.item()

        # Añadir la nueva predicción a la ventana deslizante
        prediction_window.append(current_emotion)

        # Solo cambiar la emoción si la mayoría de las últimas predicciones son consistentes
        most_common_emotion, count = Counter(prediction_window).most_common(1)[0]
        if most_common_emotion != previous_emotion and count > 2:  # Cambiar solo si hay al menos 3 predicciones consistentes
            print(f'Emoción cambiada: {most_common_emotion}')
            previous_emotion = most_common_emotion

        # Calcular y mostrar la latencia
        latency = calculate_latency(start_time)
        print(f'Latencia: {latency:.6f} segundos')

    # time.sleep(0.1)  # Ajusta según la velocidad de adquisición de características en tiempo real


Latencia: 0.005101 segundos
Latencia: 0.004250 segundos
Latencia: 0.004168 segundos
Emoción cambiada: 6
Latencia: 0.004136 segundos
Latencia: 0.003963 segundos
Latencia: 0.004017 segundos
Latencia: 0.004058 segundos
Latencia: 0.004969 segundos
Latencia: 0.003757 segundos
Latencia: 0.004573 segundos
Latencia: 0.004154 segundos
Latencia: 0.003972 segundos
Latencia: 0.004519 segundos
Latencia: 0.003834 segundos
Emoción cambiada: 1
Latencia: 0.004051 segundos
Latencia: 0.003682 segundos
Latencia: 0.004811 segundos
Latencia: 0.004623 segundos
Latencia: 0.005108 segundos
Latencia: 0.004852 segundos
Latencia: 0.003990 segundos
Latencia: 0.003938 segundos
Latencia: 0.003678 segundos
Latencia: 0.004676 segundos
Latencia: 0.003691 segundos
Latencia: 0.003723 segundos
Latencia: 0.003700 segundos
Latencia: 0.003759 segundos
Latencia: 0.003667 segundos
Latencia: 0.003426 segundos
Latencia: 0.003671 segundos
Latencia: 0.005462 segundos
Latencia: 0.004666 segundos
Latencia: 0.003667 segundos
Latencia

KeyboardInterrupt: 

In [9]:
import torch
import numpy as np
from collections import deque

# Cargar el modelo completo
model = torch.load('modelo_emociones_lstm_completo.pth')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
model.eval()  # Establecer el modelo en modo de evaluación

# Configuración de la secuencia de tiempo (timesteps)
n_timesteps = 101  # Asegúrate de que coincida con lo que espera tu modelo
n_features = model.fc.in_features  # Esto debería coincidir con el número de características por timestep

# Inicializar un buffer para mantener las secuencias de características de tiempo real
sequence_buffer = deque(maxlen=n_timesteps)

# Suponiendo que tienes un flujo de características en tiempo real (aquí usaremos una función ficticia)
def get_real_time_features():
    # Esta función debería devolver un vector de características de longitud `n_features`
    # Aquí simplemente devolveremos datos aleatorios como ejemplo
    return np.random.rand(n_features)

# Ejemplo de bucle de predicción en tiempo real
while True:
    # Obtener nuevas características en tiempo real
    new_features = get_real_time_features()

    # Añadir las nuevas características al buffer
    sequence_buffer.append(new_features)

    # Verificar si hemos llenado el buffer con suficientes timesteps
    if len(sequence_buffer) == n_timesteps:
        # Convertir el buffer a un tensor y hacer la predicción
        sequence_array = np.array(sequence_buffer)
        input_tensor = torch.tensor(sequence_array, dtype=torch.float32).unsqueeze(0).to(device)  # Añadir dimensión de batch

        with torch.no_grad():
            output = model(input_tensor)
            _, predicted = torch.max(output, 1)
            predicted_emotion = predicted.item()
            print(f'Emoción predicha: {predicted_emotion}')

        # Aquí podrías agregar una lógica para mostrar la emoción en pantalla, guardarla, etc.

    # Simulación de esperar un poco antes de la siguiente iteración (ej. 100ms)
    # Para tiempos reales, esta parte dependería de cómo obtienes las características en tiempo real
    # time.sleep(0.1)


/var/folders/pv/hj1tkdtn4xd2lk842kp9xy7r0000gn/T/ipykernel_4932/3903378472.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load('modelo_emociones_lstm_comp

RuntimeError: input.size(-1) must be equal to input_size. Expected 38, got 128

## LAST TRY


In [22]:
import librosa
import numpy as np
import torch
from collections import deque
import time  # No olvides importar time si aún no lo has hecho

# Asegúrate de que el modelo esté cargado y en modo de evaluación
model.eval()

# Configuración del buffer de secuencias
n_timesteps = 100  # Igual que en el entrenamiento
sequence_buffer = deque(maxlen=n_timesteps)

# Simulación o configuración real de la captura de audio
def capture_audio(duration=2.0, sr=22050):
    # Captura audio durante 'duration' segundos
    # En un entorno real, usarías un micrófono para capturar el audio
    y = librosa.load(librosa.example('trumpet'), duration=duration, sr=sr)[0]
    return y, sr

# Extraer características con el mismo proceso que en el entrenamiento
def extract_features_real_time(y, sr, n_timesteps=100):
    # Ajusta hop_length y n_fft para evitar el error
    hop_length = len(y) // n_timesteps
    n_fft = min(2048, len(y))  # Asegura que n_fft no sea mayor que la longitud de y
    
    # Extraer características de la misma manera que en el entrenamiento
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13, hop_length=hop_length, n_fft=n_fft)
    chroma = librosa.feature.chroma_stft(y=y, sr=sr, hop_length=hop_length, n_fft=n_fft)
    spec_contrast = librosa.feature.spectral_contrast(y=y, sr=sr, hop_length=hop_length, n_fft=n_fft)
    tonnetz = librosa.feature.tonnetz(y=librosa.effects.harmonic(y), sr=sr, hop_length=hop_length)
    
    # Combina todas las características
    features = np.hstack([mfccs.T, chroma.T, spec_contrast.T, tonnetz.T])
    return features

# Bucle de predicción en tiempo real
while True:
    y, sr = capture_audio(duration=2.0)  # Captura audio durante 2 segundos
    new_features = extract_features_real_time(y, sr, n_timesteps=n_timesteps)

    # Añadir las nuevas características al buffer
    sequence_buffer.append(new_features)

    # Verificar si el buffer tiene suficientes timesteps
    if len(sequence_buffer) == n_timesteps:
        sequence_array = np.array(sequence_buffer)
        input_tensor = torch.tensor(sequence_array, dtype=torch.float32).unsqueeze(0).to(device)

        with torch.no_grad():
            output = model(input_tensor)
            _, predicted = torch.max(output, 1)
            current_emotion = predicted.item()

        print(f'Predicción de emoción: {current_emotion}')

    # Espera un tiempo breve antes de la siguiente captura de audio
    time.sleep(0.1)


KeyboardInterrupt: 

In [27]:
import sounddevice as sd
import librosa
import numpy as np
import torch
from collections import deque

# Asegúrate de que el modelo esté cargado y en modo de evaluación
model.eval()

# Configuración del buffer de secuencias
n_timesteps = 100  # Igual que en el entrenamiento
sequence_buffer = deque(maxlen=n_timesteps)

# Configuración de la captura de audio en tiempo real
sample_rate = 22050  # Frecuencia de muestreo
duration = 2.0  # Duración de la grabación en segundos

# Función para capturar audio en tiempo real desde el micrófono
def capture_audio(duration, sample_rate):
    print("Capturando audio...")
    audio = sd.rec(int(duration * sample_rate), samplerate=sample_rate, channels=1, dtype='float32')
    sd.wait()  # Espera a que termine la grabación
    audio = np.squeeze(audio)  # Quitar la dimensión extra
    print(f"Audio capturado. Longitud: {len(audio)}")
    return audio, sample_rate

# Extraer características con el mismo proceso que en el entrenamiento
def extract_features_real_time(y, sr, n_timesteps=100):
    print("Extrayendo características...")
    hop_length = len(y) // n_timesteps
    n_fft = min(2048, len(y))  # Asegura que n_fft no sea mayor que la longitud de y
    
    # Extraer características de la misma manera que en el entrenamiento
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13, hop_length=hop_length, n_fft=n_fft)
    chroma = librosa.feature.chroma_stft(y=y, sr=sr, hop_length=hop_length, n_fft=n_fft)
    spec_contrast = librosa.feature.spectral_contrast(y=y, sr=sr, hop_length=hop_length, n_fft=n_fft)
    tonnetz = librosa.feature.tonnetz(y=librosa.effects.harmonic(y), sr=sr, hop_length=hop_length)
    
    # Combina todas las características
    features = np.hstack([mfccs.T, chroma.T, spec_contrast.T, tonnetz.T])
    print(f"Características extraídas: {features.shape}")
    return features

# Bucle de predicción en tiempo real
while True:
    y, sr = capture_audio(duration=2.0, sample_rate=sample_rate)  # Captura audio desde el micrófono
    new_features = extract_features_real_time(y, sr, n_timesteps=n_timesteps)

    # Añadir las nuevas características al buffer
    sequence_buffer.extend(new_features)
    print(f"Tamaño actual del buffer: {len(sequence_buffer)}")

    # Verificar si el buffer tiene suficientes timesteps
    if len(sequence_buffer) >= n_timesteps:
        print("Buffer lleno. Preparando la entrada para el modelo...")
        sequence_array = np.array(sequence_buffer)
        input_tensor = torch.tensor(sequence_array, dtype=torch.float32).unsqueeze(0).to(device)

        with torch.no_grad():
            output = model(input_tensor)
            _, predicted = torch.max(output, 1)
            current_emotion = predicted.item()

        print(f'Predicción de emoción: {current_emotion}')

        # Opcional: Vaciar el buffer después de la predicción
        sequence_buffer.clear()

    # Espera un tiempo breve antes de la siguiente captura de audio
    time.sleep(0.1)


Capturando audio...
Audio capturado. Longitud: 44100
Extrayendo características...


/Users/sofiaguerrero/miniconda3/lib/python3.12/site-packages/librosa/core/pitch.py:101: UserWarning: Trying to estimate tuning from empty frequency set.
  return pitch_tuning(


Características extraídas: (101, 38)
Tamaño actual del buffer: 100
Buffer lleno. Preparando la entrada para el modelo...
Predicción de emoción: 3
Capturando audio...
Audio capturado. Longitud: 44100
Extrayendo características...
Características extraídas: (101, 38)
Tamaño actual del buffer: 100
Buffer lleno. Preparando la entrada para el modelo...
Predicción de emoción: 3
Capturando audio...
Audio capturado. Longitud: 44100
Extrayendo características...
Características extraídas: (101, 38)
Tamaño actual del buffer: 100
Buffer lleno. Preparando la entrada para el modelo...
Predicción de emoción: 3
Capturando audio...
Audio capturado. Longitud: 44100
Extrayendo características...
Características extraídas: (101, 38)
Tamaño actual del buffer: 100
Buffer lleno. Preparando la entrada para el modelo...
Predicción de emoción: 3
Capturando audio...
Audio capturado. Longitud: 44100
Extrayendo características...
Características extraídas: (101, 38)
Tamaño actual del buffer: 100
Buffer lleno. Pre

KeyboardInterrupt: 

## test with wav test audio

In [43]:
import librosa
import numpy as np
import torch
from collections import deque
import soundfile as sf

# Ensure your model is loaded and in evaluation mode
model.eval()

# Load the pre-recorded audio file
audio_path = 'testAudio2.m4a'  # Replace with your file path
y, sr = librosa.load(audio_path, sr=22050)  # Load the audio file with the desired sample rate

# Set up sliding window parameters
segment_duration = 3.0  # in seconds
hop_duration = 1.0  # in seconds (overlap)
n_timesteps = 101  # Must match your training setup
sequence_buffer = deque(maxlen=n_timesteps)

# Mapeo de índices a nombres de emociones
emotion_map = {0: 'Enojo', 1: 'Disgusto', 2: 'Miedo', 3: 'Felicidad', 4: 'Tristeza', 5: 'Sorpresa', 6: 'Neutral'}

# Process the audio in overlapping segments
for start_sample in range(0, len(y) - int(segment_duration * sr), int(hop_duration * sr)):
    print(f"Processing segment starting at {start_sample / sr:.2f}s")

    # Extract the segment
    segment = y[start_sample:start_sample + int(segment_duration * sr)]
    
    # Debugging: Save the segment to inspect it
    sf.write(f'segment_{start_sample}.wav', segment, sr)
    
    # Extract features for this segment
    hop_length = len(segment) // n_timesteps
    mfccs = librosa.feature.mfcc(y=segment, sr=sr, n_mfcc=13, hop_length=hop_length)
    chroma = librosa.feature.chroma_stft(y=segment, sr=sr, hop_length=hop_length)
    spec_contrast = librosa.feature.spectral_contrast(y=segment, sr=sr, hop_length=hop_length)
    tonnetz = librosa.feature.tonnetz(y=librosa.effects.harmonic(segment), sr=sr, hop_length=hop_length)
    
    # Combine features
    features = np.hstack([mfccs.T, chroma.T, spec_contrast.T, tonnetz.T])
    
    # Add features to the buffer
    sequence_buffer.append(features)
    print(f"Tamaño actual del buffer: {len(sequence_buffer)}")  # Debugging: Check buffer size
    
    if len(sequence_buffer) == n_timesteps:
        # Convert buffer to tensor and make a prediction
        sequence_array = np.array(sequence_buffer)
        input_tensor = torch.tensor(sequence_array, dtype=torch.float32).unsqueeze(0).to(device)

        print("Input tensor shape:", input_tensor.shape)  # Debugging: Print the shape of the tensor
        print("Input tensor data:", input_tensor)  # Debugging: Print the data of the tensor

        with torch.no_grad():
            output = model(input_tensor)
            print("Model output:", output)  # Debugging: Print raw model output
            _, predicted = torch.max(output, 1)
            current_emotion = predicted.item()
            emotion_name = emotion_map[current_emotion]
            print(f'Predicted emotion: {emotion_name}')

# Optionally save the segmented audio if needed
output_audio_path = 'path/to/save/segmented/audio.wav'
sf.write(output_audio_path, y, sr)


/var/folders/pv/hj1tkdtn4xd2lk842kp9xy7r0000gn/T/ipykernel_4932/3367452084.py:12: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(audio_path, sr=22050)  # Load the audio file with the desired sample rate


Processing segment starting at 0.00s
Tamaño actual del buffer: 1
Processing segment starting at 1.00s
Tamaño actual del buffer: 2
Processing segment starting at 2.00s
Tamaño actual del buffer: 3
Processing segment starting at 3.00s
Tamaño actual del buffer: 4
Processing segment starting at 4.00s
Tamaño actual del buffer: 5
Processing segment starting at 5.00s
Tamaño actual del buffer: 6
Processing segment starting at 6.00s
Tamaño actual del buffer: 7
Processing segment starting at 7.00s
Tamaño actual del buffer: 8
Processing segment starting at 8.00s
Tamaño actual del buffer: 9
Processing segment starting at 9.00s
Tamaño actual del buffer: 10
Processing segment starting at 10.00s
Tamaño actual del buffer: 11
Processing segment starting at 11.00s
Tamaño actual del buffer: 12
Processing segment starting at 12.00s
Tamaño actual del buffer: 13
Processing segment starting at 13.00s
Tamaño actual del buffer: 14
Processing segment starting at 14.00s
Tamaño actual del buffer: 15
Processing seg

LibsndfileError: Error opening 'path/to/save/segmented/audio.wav': System error.

In [57]:
import librosa
import numpy as np
import torch
from collections import deque
import soundfile as sf

# Ensure your model is loaded and in evaluation mode
model.eval()

# Load the pre-recorded audio file
audio_path = 'testAudio2.m4a'  # Replace with your file path
y, sr = librosa.load(audio_path, sr=22050)  # Load the audio file with the desired sample rate

# Set up parameters for real-time simulation
segment_duration = 0.5  # 0.5-second segments for faster buffer filling
hop_duration = 0.25  # 0.25-second overlap
n_timesteps = 101  # Must match your training setup
sequence_buffer = deque(maxlen=n_timesteps)

# Mapeo de índices a nombres de emociones
emotion_map = {0: 'Enojo', 1: 'Disgusto', 2: 'Miedo', 3: 'Felicidad', 4: 'Tristeza', 5: 'Sorpresa', 6: 'Neutral'}

# Process the audio in smaller overlapping segments
for start_sample in range(0, len(y) - int(segment_duration * sr), int(hop_duration * sr)):
    print(f"Processing segment starting at {start_sample / sr:.2f}s")

    # Extract the segment
    segment = y[start_sample:start_sample + int(segment_duration * sr)]

    # Save the segment for debugging
    sf.write(f'segment_{start_sample}.wav', segment, sr)
    
    # Extract features for this segment
    hop_length = len(segment) // (n_timesteps - 1)
    mfccs = librosa.feature.mfcc(y=segment, sr=sr, n_mfcc=13, hop_length=hop_length)
    chroma = librosa.feature.chroma_stft(y=segment, sr=sr, hop_length=hop_length)
    spec_contrast = librosa.feature.spectral_contrast(y=segment, sr=sr, hop_length=hop_length)
    tonnetz = librosa.feature.tonnetz(y=librosa.effects.harmonic(segment), sr=sr, hop_length=hop_length)
    
    # Combine features
    features = np.hstack([mfccs.T, chroma.T, spec_contrast.T, tonnetz.T])
    
    # Add features to the buffer
    sequence_buffer.extend(features)  # Add multiple timesteps at once
    print(f"Current buffer size: {len(sequence_buffer)}")

    if len(sequence_buffer) == n_timesteps:
        # Convert buffer to tensor and make a prediction
        sequence_array = np.array(sequence_buffer)
        input_tensor = torch.tensor(sequence_array, dtype=torch.float32).unsqueeze(0).to(device)
        
        with torch.no_grad():
            output = model(input_tensor)
            _, predicted = torch.max(output, 1)
            current_emotion = predicted.item()
            emotion_name = emotion_map.get(current_emotion, "Unknown")
            print(f'Predicted emotion: {emotion_name}')

        # Clear buffer after prediction for next set of segments
        sequence_buffer.clear()


/var/folders/pv/hj1tkdtn4xd2lk842kp9xy7r0000gn/T/ipykernel_4932/795999946.py:12: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(audio_path, sr=22050)  # Load the audio file with the desired sample rate


Processing segment starting at 0.00s


/Users/sofiaguerrero/miniconda3/lib/python3.12/site-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=32768 is too large for input signal of length=5513
  warnings.warn(


Current buffer size: 101
Predicted emotion: Disgusto
Processing segment starting at 0.25s
Current buffer size: 101
Predicted emotion: Disgusto
Processing segment starting at 0.50s
Current buffer size: 101
Predicted emotion: Disgusto
Processing segment starting at 0.75s
Current buffer size: 101
Predicted emotion: Disgusto
Processing segment starting at 1.00s
Current buffer size: 101
Predicted emotion: Disgusto
Processing segment starting at 1.25s
Current buffer size: 101
Predicted emotion: Disgusto
Processing segment starting at 1.50s
Current buffer size: 101
Predicted emotion: Neutral
Processing segment starting at 1.75s
Current buffer size: 101
Predicted emotion: Neutral
Processing segment starting at 2.00s
Current buffer size: 101
Predicted emotion: Neutral
Processing segment starting at 2.25s
Current buffer size: 101
Predicted emotion: Miedo
Processing segment starting at 2.50s
Current buffer size: 101
Predicted emotion: Felicidad
Processing segment starting at 2.75s
Current buffer 

KeyboardInterrupt: 